In [1]:
from deepface.detectors import FaceDetector
import pandas as pd
import cv2
import os
from pathlib import Path
from contextlib import redirect_stdout
from time import time
import tarfile

2022-11-10 11:55:18.596259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface']
scores = pd.DataFrame(columns=['single_person', 'two_person', 'multiple_person', 'latency'], index=backends)
scores.loc[:,'latency'] = 0

/var/folders/th/8x8sv1317rl_27mws2jf40980000gn/T/ipykernel_2546/468426028.py:3: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  scores.loc[:,'latency'] = 0


In [8]:
if not os.path.exists('images/'):
    with tarfile.open('images.tar.gz', "r:gz") as f:
        f.extractall()

In [10]:
image = cv2.imread('images/single_person/1_1.jpeg')

In [13]:
for d in ['single_person', 'two_person', 'multiple_person']:
    directory = Path(f'images/{d}')
    for b in backends:
        faces_detected, faces = 0,0
        for ims in os.listdir(directory):
            if ims.startswith('.'):
                continue
            image = cv2.imread(str(directory) + '/' + ims)
            with open(os.devnull, 'w') as devnull:
                with redirect_stdout(devnull):
                    start = time()
                    out = FaceDetector.detect_faces(FaceDetector.build_model(b), b, image)
                    end = time()
            scores.loc[b, 'latency'] += ((end - start) * 1000)
            faces_detected += len(out)
            faces += int(ims.split('.')[0].split('_')[-1])
        scores.loc[b, d] = faces_detected/faces
scores.loc[:,'latency'] /= 15

2022-11-10 12:03:40.065669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


single_person, two_person, and multiple_person columns show detection accuracy (number of people detected / number of people)
latency is the average latency of the model in ms

In [14]:
scores

,single_person,two_person,multiple_person,latency
opencv,0.6,0.2,0.259259,18.005308
ssd,0.8,0.5,0.555556,30.898984
dlib,0.8,0.4,0.62963,63.737583
mtcnn,1.0,0.8,0.962963,570.498308
retinaface,1.0,0.9,1.0,2029.209614
